## Advent of Code

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation X|Y means that if both page number X and page number Y are to be produced as part of an update, page number X must be printed at some point before page number Y.

The Elf has for you both the page ordering rules and the pages to produce in each update (your puzzle input), but can't figure out whether each update has the pages in the right order.

For example:

In [ ]:
from fastcore.utils import L,first
from aocd import get_data
inp = get_data(day=5, year=2024)
samp = '''47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47'''
print(samp)

47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47


The first section specifies the page ordering rules, one per line. The first rule, 47|53, means that if an update includes both page number 47 and page number 53, then page number 47 must be printed at some point before page number 53. (47 doesn't necessarily need to be immediately before 53; other pages are allowed to be between them.)

The second section specifies the page numbers of each update. Because most safety manuals are different, the pages needed in the updates are different too. The first update, 75,47,61,53,29, means that the update consists of page numbers 75, 47, 61, 53, and 29.

To get the printers going as soon as possible, start by identifying which updates are already in the right order.

In the above example, the first update (75,47,61,53,29) is in the right order:

- 75 is correctly first because there are rules that put each other page after it: 75|47, 75|61, 75|53, and 75|29.
- 47 is correctly second because 75 must be before it (75|47) and every other page must be after it according to 47|61, 47|53, and 47|29.
- 61 is correctly in the middle because 75 and 47 are before it (75|61 and 47|61) and 53 and 29 are after it (61|53 and 61|29).
- 53 is correctly fourth because it is before page number 29 (53|29).
- 29 is the only page left and so is correctly last.
Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

The second and third updates are also in the correct order according to the rules. Like the first update, they also do not include every page number, and so only some of the ordering rules apply - within each update, the ordering rules that involve missing page numbers are not used.

The fourth update, 75,97,47,61,53, is not in the correct order: it would print 75 before 97, which violates the rule 97|75.

The fifth update, 61,13,29, is also not in the correct order, since it breaks the rule 29|13.

The last update, 97,13,75,29,47, is not in the correct order due to breaking several rules.

For some reason, the Elves also need to know the middle page number of each update being printed. Because you are currently only printing the correctly-ordered updates, you will need to find the middle page number of each correctly-ordered update. In the above example, the correctly-ordered updates are:

```
75,47,61,53,29
97,61,53,29,13
75,29,13
```

These have middle page numbers of 61, 53, and 29 respectively. Adding these page numbers together gives 143.

Of course, you'll need to be careful: the actual list of page ordering rules is bigger and more complicated than the above example.

Determine which updates are already in the correct order. What do you get if you add up the middle page number from those correctly-ordered updates?

Can you help me get started with this problem?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

I'd be happy to help you get started with this Advent of Code problem! Let me break down what we need to do:

**Understanding the problem:**
1. We have ordering rules (like `47|53` means 47 must come before 53)
2. We have updates (lists of page numbers)
3. We need to check which updates follow all the relevant rules
4. For valid updates, we find the middle page number and sum them

**First steps:**
Before we code anything, let's think about the approach. What do you think would be a good way to:
- Parse the input into rules and updates?
- Check if a single update is valid according to the rules?

Do you have initial thoughts on how you'd tackle either of these parts?

I can do the parsing. I'm not sure how to check the updates.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let me help you think through checking the updates.

For a given update like `75,47,61,53,29`, you need to verify it doesn't violate any rules. 

Here's a key insight: if you have a rule `X|Y` (X must come before Y), this rule is only violated if **both** X and Y appear in the update **and** Y comes before X.

So one approach could be:
- For each rule `X|Y`, check if both X and Y are in the current update
- If they are, check their positions - is X's position less than Y's position?
- If any rule is violated, the update is invalid

Does this make sense? Would you like to try coding this check, or would you like me to clarify anything first?

In [ ]:
def split_on(lst, delimiter):
    result = []
    current = []
    for item in lst:
        if item == delimiter:
            if current:
                result.append(current)
                current = []
        else:
            current.append(item)
    if current:
        result.append(current)
    return result

In [ ]:
xs = samp.splitlines()
rules_txt, updates_txt = split_on(xs, '')
rules_txt, updates_txt

(['47|53',
  '97|13',
  '97|61',
  '97|47',
  '75|29',
  '61|13',
  '75|53',
  '29|13',
  '97|29',
  '53|29',
  '61|53',
  '97|53',
  '61|29',
  '47|13',
  '75|47',
  '97|75',
  '47|61',
  '75|61',
  '47|29',
  '75|13',
  '53|13'],
 ['75,47,61,53,29',
  '97,61,53,29,13',
  '75,29,13',
  '75,97,47,61,53',
  '61,13,29',
  '97,13,75,29,47'])

In [ ]:
def to_num_list(s, delim): return L(s.split(delim)).map(int)

In [ ]:
from fastcore.utils import tuplify

rules = L(rules_txt).map(to_num_list, delim='|').map(tuplify)
rules

(#21) [(47, 53),(97, 13),(97, 61),(97, 47),(75, 29),(61, 13),(75, 53),(29, 13),(97, 29),(53, 29),(61, 53),(97, 53),(61, 29),(47, 13),(75, 47),(97, 75),(47, 61),(75, 61),(47, 29),(75, 13)...]

In [ ]:
updates = L(updates_txt).map(to_num_list, delim=',')
updates

(#6) [[75, 47, 61, 53, 29],[97, 61, 53, 29, 13],[75, 29, 13],[75, 97, 47, 61, 53],[61, 13, 29],[97, 13, 75, 29, 47]]

In [ ]:
u = updates[0]
s = set(u)
rules.filter(lambda t: t[0] in s and t[1] in s)

(#10) [(47, 53),(75, 29),(75, 53),(53, 29),(61, 53),(61, 29),(75, 47),(47, 61),(75, 61),(47, 29)]

How can I get index of an item in a list?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can use the `.index()` method on a list. For example:

```python
u.index(47)  # returns the position of 47 in the list u
```

This will give you the index (position) of the first occurrence of that item in the list.

Does that help you move forward with checking the rules?

In [ ]:
u.index(47), u.index(53)

(1, 3)

In [ ]:
def is_violated(rule, update):
    return update.index(rule[0]) >= update.index(rule[1])

In [ ]:
def is_in_order(update, rules):
    s = set(update)
    return not any(rules.filter(lambda t: t[0] in s and t[1] in s).filter(is_violated, update=update))

updates.map(is_in_order, rules=rules)

(#6) [True,True,True,False,False,False]

In [ ]:
def get_middle(x): return x[int(len(x)/2)]

updates.filter(is_in_order, rules=rules).map(get_middle)

(#3) [61,53,29]

In [ ]:
def solve(lines):
    rules_txt, updates_txt = split_on(lines, '')
    rules = L(rules_txt).map(to_num_list, delim='|').map(tuplify).
    updates = L(updates_txt).map(to_num_list, delim=',')
    return updates.filter(is_in_order, rules=rules).map(get_middle).sum()

solve(xs)

143

In [ ]:
solve(inp.splitlines())

4774

# Part 2

While the Elves get to work printing the correctly-ordered updates, you have a little time to fix the rest of them.

For each of the incorrectly-ordered updates, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:

- 75,97,47,61,53 becomes 97,75,47,61,53.
- 61,13,29 becomes 61,29,13.
- 97,13,75,29,47 becomes 97,75,47,29,13.

After taking only the incorrectly-ordered updates and ordering them correctly, their middle page numbers are 47, 29, and 47. Adding these together produces 123.

Find the updates which are not in the correct order. What do you get if you add up the middle page numbers after correctly ordering just those updates?

Idea:
- Each incorrectly ordered update will need to be reordered.
- As in the part 1, let's filter only those rules which have numbers in the update.

Can you give me a hint for the part 2?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Nice work on part 1! 

For part 2, you need to reorder the incorrectly-ordered updates. Here's a hint:

Think about what the rules tell you - if you have a rule `X|Y`, it means X must come before Y. This is essentially defining a **partial ordering** of the pages.

One approach: for a given update with its relevant rules, you could sort the pages based on these ordering constraints. How many rules say a particular page must come *before* other pages in that update? Or conversely, how many pages must come *after* it?

Does this spark any ideas for how you might reorder them?

I'm not sure how to reorder them.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let me give you a more concrete hint!

Python's `sorted()` function has a `key` parameter that lets you specify how to compare items. For this problem, you could use it to sort the pages in an update.

Here's the key idea: when sorting two pages `a` and `b`, you need to decide which comes first. Check if there's a rule `(a, b)` in your filtered rules - if so, `a` should come before `b`.

You could write a small function that takes two pages and returns whether the first should come before the second based on the rules. Then use that with sorting.

Does that give you enough to try implementing it?

In [ ]:
def comp(x, y, rules):
    if (x, y) in rules:
        return -1
    elif (y, x) in rules:
        return 1
    else:
        return 0

comp(47, 53, rules), comp(53, 47, rules), comp(29, 47, rules)


(-1, 1, 1)

In [ ]:
updates[3]

(#5) [75,97,47,61,53]

In [ ]:
from functools import cmp_to_key
sorted(updates[3], key=cmp_to_key(lambda x, y: comp(x, y, rules)))

[97, 75, 47, 61, 53]

In [ ]:
updates.filter(is_in_order, rules=rules)

(#3) [[75, 47, 61, 53, 29],[97, 61, 53, 29, 13],[75, 29, 13]]

In [ ]:
def is_not_in_order(update, rules): return not is_in_order(update, rules)

In [ ]:
def solve_part2(lines):
    rules_txt, updates_txt = split_on(lines, '')
    rules = L(rules_txt).map(to_num_list, delim='|').map(tuplify)
    updates = L(updates_txt).map(to_num_list, delim=',')
    cmp_fn = cmp_to_key(lambda x, y: comp(x, y, rules))
    return updates.filter(is_not_in_order, rules=rules).map(sorted, key=cmp_fn).map(get_middle).sum()

solve_part2(xs)

123

In [ ]:
solve_part2(inp.splitlines())

6004

Can you reflect on the solution and consider improvements?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great job solving both parts! Let me reflect on your solution:

**What works well:**
- Your use of `cmp_to_key` with a custom comparator is a clean approach for sorting with rules
- The functional style with `.filter()` and `.map()` makes the logic easy to follow
- You correctly identified that rules act as a partial ordering

**Potential improvements to consider:**

1. **Efficiency concern:** Your `comp` function checks if tuples are in the full `rules` list. For part 2, you could filter rules per update (like you did conceptually in part 1) to only check relevant rules. Would this make lookups faster?

2. **Edge case:** What happens if `comp` returns 0 (no rule exists between two pages)? Does the sort still work correctly? 

3. **Code reuse:** You're parsing the input twice in `solve` and `solve_part2`. Could you refactor to parse once?

Which of these (if any) interests you to explore further?